In [1]:
import os
from pathlib2 import Path
home = Path.cwd()
dest = Path(str(home.parents[1]) + '\\gis')
os.chdir(dest)
import esri
from arcgis.features import FeatureLayer
import arcgis.geoenrichment
from arcgis.geoenrichment import enrich
os.chdir(home)
import pandas as pd
import json

C:\Users\Chris_D\Anaconda3\lib\site-packages\arcgis\features\_data\geodataset\utils.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


In [2]:
def countries(cntry = None):
    '''Returns all countries if no args. Returns country if country arg.'''
    return arcgis.geoenrichment.Country.get(cntry) if cntry else arcgis.geoenrichment.get_countries()

def cbsa(sg, sa = 'metropolitan'):
    '''Returns dict of metropolitan or micropolitan for statistical area chosen for specified subgeography
    and specified statistical area. Default is metropolitan.'''
    return {k: v for (k, v) in sg.subgeographies.cbsa.items() if sa in k.lower()}

def area_ids(d):
    '''Returns a dict of area_id codes to named areas. User enters dict of named areas.'''
    codes_nas = {}
    for k, v in d.items():
        vals = str(v).split()
        for x in vals:
            if 'area_id' in x:
                aid, code = x.split('=')
                #Removing leading/trailing punctuation.
                code = code[1:-2]
                codes_nas[code] = v
    return codes_nas

def credit_calc(atts, dps):
    '''Calculates number of credits used in ArcGIS online for geoenrichment. User enters number of attributes
    that need to be enriched and the number geoenrichment data points.'''
    return (atts / 1000) * 10 * dps

In [3]:
#Have to login to access geoenrichment.
l = esri.Arcgis(str(dest) + '\\config.json')

In [4]:
usa = countries('US')
metros = cbsa(usa)
data = usa.data_collections

In [6]:
#Codes and named areas since cbsa id codes are embedded in named area and don't know how to derive property.
na_codes = area_ids(metros)

In [7]:
#Reading in geolocated records with metro code information in them.
file_dir = '\\'.join(str(dest).split('\\')[0:-1] + ['levels'])
os.chdir(file_dir)
geoc = pd.read_csv('10_11_20_geoc_metros.csv')

C:\Users\Chris_D\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
#Removing values without a cbsa code.
geoc = geoc[~geoc.loc[:, 'CBSA Code'].isnull()]
#Changing datatype.
geoc.loc[:, 'CBSA Code'] = geoc.loc[:, 'CBSA Code'].astype('int64').astype(str)

In [9]:
#Removing any named areas in the na_codes dict that do not exist in my df.
na_codes = {k: v for (k, v) in na_codes.items() if k in geoc.loc[:, 'CBSA Code'].unique()}

In [10]:
#Seems like 2 codes are not in the arcgis named areas.
len(na_codes) #93
len(geoc.loc[:, 'CBSA Code'].unique()) #95
#2 records for Puerto Rico. Must have been entered in with CBSA codes after the last update.
geoc[~geoc.loc[:, 'CBSA Code'].isin(na_codes)]
#Removing those two records from the dataset.
geoc = geoc[~geoc.index.isin(geoc[~geoc.loc[:, 'CBSA Code'].isin(na_codes)].index)]

In [11]:
#Reading in variables to fetch from ArcGIS content.
with open(Path.joinpath(home, 'variables.json')) as f:
    ag_vars = json.load(f)

In [12]:
#Study areas.
study_ars = list(na_codes.values())

# Enriching data and writing data in parts to avoid accidental credit overusage.

In [25]:
path = Path.joinpath(Path.cwd(), 'geoenrich', 'enriched')

In [19]:
# edu = enrich(study_areas = study_ars, analysis_variables = ag_vars['educ'])
edu.to_csv(Path.joinpath(path, 'educ.csv'), index = False)

In [35]:
# diverse = enrich(study_areas = study_ars, analysis_variables = ag_vars['div'])
diverse.to_csv(Path.joinpath(path, 'diversity.csv'), index = False)

In [42]:
# unemp = enrich(study_areas = study_ars, analysis_variables = ag_vars['unem'])
unemp.to_csv(Path.joinpath(path, 'unemployment.csv'), index = False)

In [58]:
# naics = enrich(study_areas = study_ars, analysis_variables = ag_vars['bus'])
naics.to_csv(Path.joinpath(path, 'businesses.csv'), index = False)

In [65]:
# ent = enrich(study_areas = study_ars, analysis_variables = ag_vars['ent'])
ent.to_csv(Path.joinpath(path, 'entertainment.csv'), index = False)

In [67]:
# crime = enrich(study_areas = study_ars, analysis_variables = ag_vars['crm'])
crime.to_csv(Path.joinpath(path, 'crime.csv'), index = False)

In [73]:
# gens = enrich(study_ars, analysis_variables = ag_vars['gens'])
gens.to_csv(Path.joinpath(path, 'generations.csv'), index = False)

In [76]:
# house = enrich(study_areas = study_ars, analysis_variables = ag_vars['house'])
house.to_csv(Path.joinpath(path, 'housing.csv'), index = False)

In [83]:
# occ = enrich(study_areas = study_ars, analysis_variables = ag_vars['occ'])
occ.to_csv(Path.joinpath(path, 'occupations.csv'), index = False)

# Reading geoenriched information back in to normalize.

In [155]:
dfs = {f.split('.')[0]: pd.read_csv(Path.joinpath(path, f)) for f in os.listdir(Path.joinpath(path)) if f.endswith('.csv')}

In [1]:
#Evaluating unique records.

# for df in dfs.values():
#     print(df.HasData.unique()) #All have ones, indicating data is there for all data.
#     print(df.sourceCountry.unique()) #All US
#     print(df.aggregationMethod.unique()) #All CBSA
#     print(df.populationToPolygonSizeRating.unique()) #All 2.191. Dont know what that means.
#     print(len(df)) #All 93 rows.
#     print(df.columns)

In [181]:
succinct = {k: pd.concat([v.iloc[0:, 3:5], v.iloc[0:, v.columns.get_loc('HasData') + 1:]], axis = 1) for (k, v) in dfs.items()}

In [183]:
#Can actually get rid of shape too...keeping for only one df since geometries are the same.
from functools import reduce
#Beginning result column names.
c_beg = ['StdGeographyID', 'StdGeographyName']
rollup = reduce(lambda df1, df2 : pd.merge(df1, df2, on = c_beg[0]), succinct.values())
#Getting rid of redundant columns.
rollup = rollup.drop([f'{c_beg[1]}_x', f'{c_beg[1]}_y', 'SHAPE_x', 'SHAPE_y'], axis = 1)
#Reordering columns.
cols = c_beg + [col for col in rollup.columns if col not in c_beg]
rollup = rollup[cols]

In [190]:
rollup.loc[:, 'StdGeographyID'] = rollup.loc[:, 'StdGeographyID'].astype(str)

In [191]:
# rollup.to_csv(Path.joinpath(path, 'all.csv'), index = False)

In [319]:
to_visualize = pd.merge(geoc, rollup, left_on = 'CBSA Code', right_on = 'StdGeographyID')

In [320]:
cbsa_url = 'https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/usa_cbsa/FeatureServer/0'
cbsa_fl = FeatureLayer(cbsa_url)

In [321]:
cbsa_features = cbsa_fl.query(where = "CBSA_TYPE = 'Metropolitan'")
#Creating spatial dataframe.
cbsa_df = cbsa_features.sdf

In [322]:
#Segmenting out poughkeepsie prior to join since code not represented in CBSA metro df.
poughkeepsie = to_visualize[to_visualize.loc[:, 'CBSA Code'] == '39100']
to_visualize = to_visualize[to_visualize.loc[:, 'CBSA Code'] != '39100']

In [323]:
poughkeepsie.loc[:, 'CBSA Code'] = '35620'

In [324]:
#Bringing back in to original df.
to_visualize = pd.concat([to_visualize, poughkeepsie], axis = 0)

In [325]:
to_visualize = pd.merge(to_visualize, cbsa_df[['CBSA_ID', 'SQMI']], left_on = 'CBSA Code', right_on = 'CBSA_ID')

In [326]:
#Identifying coolumns that need to be divided by square mile.
vars_psqm = list({k: v for (k, v) in ag_vars.items() if k not in ['crm', 'ent', 'pop', 'disp', 'ind_emp']}.values())
vars_psqm = [i for l in vars_psqm for i in l]
#Removing index items that were embedded in original lists.
vars_psqm.remove('DIVINDX_CY')
vars_psqm.remove('HAI_CY')

In [327]:
#Creating per square mile calculated columns for appropriate columns.
for c in vars_psqm:
    to_visualize.loc[:, c + '_sqmi'] = to_visualize.loc[:, c] / to_visualize.loc[:, 'SQMI']

In [343]:
#Dropping SHAPE (geometry) column for now for ease of viewing tabular data.
to_visualize = to_visualize.drop('SHAPE', axis = 1)

In [348]:
to_visualize.to_csv('10_11_20_enriched.csv', index = False)